In [1]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from sklearn.model_selection import train_test_split
from mSDA.msda_theano import mSDATheano
import theano.tensor as T

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[1]

# Pruebas con el dataset Amazon (3000 Dimensiones)

In [2]:
dims = 3000
dataset_name = datasets[0]

In [3]:
print tipo
print dataset_name
print dims
print data_path

msda
amazon
3000
data


In [4]:
# cargando dataset
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [5]:
parametros = {
    'noises': [0.3, 0.5, 0.8],
    'layers': [1, 3, 5]
}

noise_layer_pair =  list(itertools.product(parametros['noises'], parametros['layers']))

x = T.dmatrix('x')

In [6]:
porcentajes = [25, 50, 100]
modelos = {}

pairs = list(itertools.permutations(domains, 2))

for src, tgt in pairs:
    modelos[(src, tgt)] = {}
    for ptje in porcentajes:
        modelos[(src, tgt)][ptje] = 100

In [7]:
df = pd.DataFrame(columns=dataframe_columns+['Porcentaje'])

i=0
j=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

porcentajes = [25, 50, 100]


for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
    tarea = src[0]+'->'+tgt[0]
    
    #datos de entrenamiento
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
    
    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tgt = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
    
    X_train = np.concatenate((X_src, X_tgt))
    
    #datos de prueba
    X_tgt_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_tgt_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
    
    
    #baseline in-domain error
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
    
    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tgt, y_tgt)
    b_error = 1-svc.score(X_tgt_ts, y_tgt_ts)

    k = 0    
    for noise, layer in noise_layer_pair:
        print "\tEntrenando modelo de adaptacion %d" % k
        # entrenar un modelo de adaptacion
        modelo_temp = mSDATheano(x, layer, noise)
        t_adaptar = modelo_temp.fit(X_train)

        # entrenar un clasificador con los datos adaptados
        X_src_a = modelo_temp.predict(X_src)
        clf = get_best_score(X_src_a, y_src, classifier='SVC', n_jobs=4)

        # se prueba con distintos porcentajes
        j = i*3
        for porcentaje in porcentajes:
            if porcentaje == 100:
                X_test_pc, y_test_pc = X_tgt_ts, y_tgt_ts
            else:
                X_test_pc, _, y_test_pc, _ = train_test_split(X_tgt_ts, y_tgt_ts, train_size=porcentaje/float(100))
            
            X_test_pc_a = modelo_temp.predict(X_test_pc)
            t_error = 1-clf.score(X_test_pc_a, y_test_pc)
            
            if t_error <= modelos[(src, tgt)][porcentaje]:
                modelos[(src, tgt)][porcentaje] = t_error
                t_loss = t_error - b_error
                df.loc[j] = ['mSDA', tarea, src, tgt, b_error*100, t_error*100, t_loss*100, porcentaje]
    
            j += 1
        k += 1
    i+=1

Tarea 1 de 12
Cargando modelo existente.
	Entrenando modelo de adaptacion 0
	Entrenando capa  1
	Entrenado en 2.41m

	Entrenando modelo de adaptacion 1
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 6.89m



/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	Entrenando modelo de adaptacion 2
	Entrenando capa  1
	Entrenado en 2.28m

	Entrenando modelo de adaptacion 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 7.01m

	Entrenando modelo de adaptacion 4
	Entrenando capa  1
	Entrenado en 2.26m

	Entrenando modelo de adaptacion 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 6.96m

Tarea 2 de 12
Cargando modelo existente.
	Entrenando modelo de adaptacion 0
	Entrenando capa  1
	Entrenado en 2.32m

	Entrenando modelo de adaptacion 1
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 6.84m

	Entrenando modelo de adaptacion 2
	Entrenando capa  1
	Entrenado en 2.30m

	Entrenando modelo de adaptacion 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 6.85m

	Entrenando modelo de adaptacion 4
	Entrenando capa  1
	Entrenado en 2.30m

	Entrenando modelo de adaptacion 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 6.95m


In [8]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss,Porcentaje
0,mSDA,e->d,electronics,dvd,15.762894,20.560000,4.797106,25
1,mSDA,e->d,electronics,dvd,15.762894,24.311881,8.548987,50
2,mSDA,e->d,electronics,dvd,15.762894,25.008125,9.245231,100
3,mSDA,e->k,electronics,kitchen,8.587715,10.216346,1.628631,25
4,mSDA,e->k,electronics,kitchen,8.587715,9.805689,1.217974,50
5,mSDA,e->k,electronics,kitchen,8.587715,11.260282,2.672567,100
6,mSDA,e->b,electronics,books,15.157879,19.791667,4.633788,25
7,mSDA,e->b,electronics,books,15.157879,23.272327,8.114448,50
8,mSDA,e->b,electronics,books,15.157879,23.815595,8.657716,100
9,mSDA,d->e,dvd,electronics,11.127778,24.235105,13.107326,25


In [9]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "inductive_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/msda/inductive_3000.csv
Resultados guardados.


# Pruebas con el dataset Twitter (2000 Dimensiones)

In [ ]:
dims = 2000
dataset_name = datasets[1]

In [ ]:
print tipo
print dataset_name
print dims
print data_path

In [ ]:
# cargando dataset
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

In [ ]:
parametros = {
    'noises': [0.3, 0.5, 0.8],
    'layers': [1, 3, 5]
}

noise_layer_pair =  list(itertools.product(parametros['noises'], parametros['layers']))

x = T.dmatrix('x')

In [ ]:
porcentajes = [25, 50, 100]
modelos = {}

pairs = list(itertools.permutations(domains, 2))

for src, tgt in pairs:
    modelos[(src, tgt)] = {}
    for ptje in porcentajes:
        modelos[(src, tgt)][ptje] = 100

In [ ]:
df = pd.DataFrame(columns=dataframe_columns+['Porcentaje'])

i=0
j=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

porcentajes = [25, 50, 100]


for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
    tarea = src[0]+'->'+tgt[0]
    
    #datos de entrenamiento
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
    
    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tgt = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
    
    X_train = np.concatenate((X_src, X_tgt))
    
    #datos de prueba
    X_tgt_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_tgt_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
    
    
    #baseline in-domain error
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
    
    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tgt, y_tgt)
    b_error = 1-svc.score(X_tgt_ts, y_tgt_ts)

    k = 0    
    for noise, layer in noise_layer_pair:
        print "\tEntrenando modelo de adaptacion %d" % k
        # entrenar un modelo de adaptacion
        modelo_temp = mSDATheano(x, layer, noise)
        t_adaptar = modelo_temp.fit(X_train)

        # entrenar un clasificador con los datos adaptados
        X_src_a = modelo_temp.predict(X_src)
        clf = get_best_score(X_src_a, y_src, classifier='SVC', n_jobs=4)

        # se prueba con distintos porcentajes
        j = i*3
        for porcentaje in porcentajes:
            if porcentaje == 100:
                X_test_pc, y_test_pc = X_tgt_ts, y_tgt_ts
            else:
                X_test_pc, _, y_test_pc, _ = train_test_split(X_tgt_ts, y_tgt_ts, train_size=porcentaje/float(100))
            
            X_test_pc_a = modelo_temp.predict(X_test_pc)
            t_error = 1-clf.score(X_test_pc_a, y_test_pc)
            
            if t_error <= modelos[(src, tgt)][porcentaje]:
                modelos[(src, tgt)][porcentaje] = t_error
                t_loss = t_error - b_error
                df.loc[j] = ['mSDA', tarea, src, tgt, b_error*100, t_error*100, t_loss*100, porcentaje]
    
            j += 1
        k += 1
    i+=1

In [ ]:
df

In [ ]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "inductive_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."